## Taller N°1 : Aplicacion de Algoritmos de B´usqueda

* Profesor: Jorge Ivan Padilla Buritica
* Estudiante: Juan Felipe Cardona Arango 
* Maestria en Ciencia de Datos y Analitica
* Curso Fundamentos de Inteligencia Artificial
* 2025-02


## Preparacion del entorno de trabajo

In [1]:
import math
import heapq
import time
from time import time

# Punto 1:Optimizacion de rutas rurales (Logistica)

## Pregunta orientadora
**¿Cómo llego del nodo de inicio al nodo final gastando lo menos posible?**

---

## Algoritmos usados
- **UCS (Búsqueda de Costo Uniforme)**: explora rutas siempre expandiendo el camino más barato encontrado hasta ahora. Garantiza el camino óptimo si los pesos son positivos.
- **A***: es parecido a UCS, pero usa una **heurística** que le da una idea de qué tan cerca está del destino. Esto le permite ser más rápido sin perder la exactitud si la heurística está bien diseñada.

---

## ¿Qué es la heurística?
Una heurística es una **estimación** del costo que falta para llegar al destino. Por ejemplo, si conocemos las coordenadas de cada punto, podemos usar la **distancia en línea recta** (distancia euclidiana) como estimación.

> Si la línea recta siempre es igual o menor que cualquier ruta real, no se sobreestima el costo y A* seguirá encontrando la mejor ruta.

---

## Solución propuesta
1. Representar el mapa como un grafo con nodos, conexiones y pesos.
2. Elegir el nodo inicial y el nodo destino.
3. Seleccionar el algoritmo (UCS o A*) y, si es A*, definir la heurística.
4. Explorar el grafo paso a paso, comparando costos acumulados.
5. Cuando se llega al destino, devolver el camino encontrado y su costo total.

---

## Medición
Para evaluar un algoritmo no basta con que encuentre el camino óptimo; también importa **el esfuerzo que le costó encontrarlo**.

Por eso medimos:
- **Costo total** del camino encontrado.
- **Nodos expandidos** durante la búsqueda.
- **Profundidad** de la solución (número de pasos o aristas).
- **Tiempo de ejecución** (en milisegundos).



## Pseudocodigo

### UCS

Entrada: grafo G, inicio s, objetivo t, función de costo w(u,v) > 0
Salida: camino óptimo s→t (si existe) y su costo g*(t)

1  open ← PriorityQueue()                      // Cola por costo acumulado g(n)
2  open.push(s, key=0)
3  g[s] ← 0                                     // Costo al inicio es 0
4  parent[s] ← None                             // Para reconstruir el camino
5  depth[s] ← 0
6  visited ← ∅                                  // Conjunto de nodos ya extraídos (cerrados)
7  expanded ← 0                                  // Métrica: conteo de expansiones
8  while open ≠ ∅:
9      (n, _) ← open.pop_min()                  // Extraer nodo con menor g(n)
10     if n ∈ visited:                          // Si ya fue cerrado, saltar
11         continue
12     visited ← visited ∪ {n}
13     expanded ← expanded + 1
14     if n == t:                                // ¡Objetivo alcanzado!
15         return Reconstruir(parent, t), g[t], expanded, depth[t]
16     for cada m ∈ neighbors(n):                // Explorar vecinos
17         costo_tentativo ← g[n] + w(n,m)       // Relajar arista (n,m)
18         if m ∉ g or costo_tentativo < g[m]:   // ¿mejor camino a m?
19             g[m] ← costo_tentativo
20             parent[m] ← n
21             depth[m] ← depth[n] + 1
22             open.push_or_decrease(m, key=g[m])
23  return ∅, ∞, expanded, -1  


### A* 

Entrada: grafo G, inicio s, objetivo t, costo w(u,v) > 0, heurística h(n)
Salida: camino óptimo s→t (si h es admisible y consistente) y su costo g*(t)

1  open ← PriorityQueue()                       // Cola por f(n)=g(n)+h(n)
2  open.push(s, key=h(s))
3  g[s] ← 0
4  parent[s] ← None
5  depth[s] ← 0
6  closed ← ∅                                    // Nodos ya expandidos (cerrados)
7  expanded ← 0
8  while open ≠ ∅:
9      (n, _) ← open.pop_min()                   // Extraer el de menor f
10     if n ∈ closed:                            // Evitar re-procesar cerrados
11         continue
12     closed ← closed ∪ {n}
13     expanded ← expanded + 1
14     if n == t:                                 // Objetivo alcanzado
15         return Reconstruir(parent, t), g[t], expanded, depth[t]
16     for cada m ∈ neighbors(n):                 // Explorar vecinos
17         costo_tentativo ← g[n] + w(n,m)        // Relajar (n,m)
18         if m ∉ g or costo_tentativo < g[m]:    // ¿mejor g para m?
19             g[m] ← costo_tentativo
20             parent[m] ← n
21             depth[m] ← depth[n] + 1
22             f_m ← g[m] + h(m)                  // Clave f para la cola
23             open.push_or_decrease(m, key=f_m)
24  return ∅, ∞, expanded, -1

## Codigo

## Implementacion

In [2]:
from __future__ import annotations
from typing import Dict, Tuple, Callable, List, Any
import math
import time
import heapq
import networkx as nx


In [3]:
Position = Tuple[float, float]
Heuristic = Callable[[Any], float]

class PriorityQueue:
    def __init__(self):
        self._pq = []
        self._entry_finder = {}
        self._REMOVED = object()
        self._counter = 0
    def push_or_decrease(self, item, priority: float):
        if item in self._entry_finder:
            # marca entrada anterior como removida
            old = self._entry_finder.pop(item)
            old[-1] = self._REMOVED
        entry = [priority, self._counter, item]
        self._counter += 1
        self._entry_finder[item] = entry
        heapq.heappush(self._pq, entry)
    def pop_min(self):
        while self._pq:
            priority, _, item = heapq.heappop(self._pq)
            if item is not self._REMOVED:
                self._entry_finder.pop(item, None)
                return item, priority
        raise KeyError('pop from an empty priority queue')
    def empty(self):
        return not self._entry_finder

class SearchResult:
    def __init__(self, path: List[Any], cost: float, expanded: int, depth: int, runtime_ms: float):
        self.path = path
        self.cost = cost
        self.expanded = expanded
        self.depth = depth
        self.runtime_ms = runtime_ms
    def as_dict(self):
        return {
            'path': self.path,
            'cost': self.cost,
            'expanded': self.expanded,
            'depth': self.depth,
            'runtime_ms': round(self.runtime_ms, 3)
        }

# -----------------
# Heurísticas
# -----------------

def h_zero(goal: Any) -> Heuristic:
    return lambda n: 0.0

def h_euclid(G: nx.Graph, goal: Any) -> Heuristic:
    xg, yg = G.nodes[goal]['pos']
    def _h(n):
        x, y = G.nodes[n]['pos']
        return math.hypot(x - xg, y - yg)
    return _h

# Escalada controlada (para estudio de sobreestimación)
# h'(n) = alpha * h_euclid(n), con alpha >= 1

def h_scaled(G: nx.Graph, goal: Any, alpha: float = 1.2) -> Heuristic:
    base = h_euclid(G, goal)
    return lambda n: alpha * base(n)

# -----------------
# Reconstrucción de camino
# -----------------

def reconstruct(parent: Dict[Any, Any], goal: Any):
    path = []
    n = goal
    while n is not None:
        path.append(n)
        n = parent.get(n, None)
    path.reverse()
    return path

# -----------------
# UCS
# -----------------

def ucs(G: nx.Graph, start: Any, goal: Any) -> SearchResult:
    t0 = time.perf_counter()
    openq = PriorityQueue()
    openq.push_or_decrease(start, 0.0)
    g = {start: 0.0}
    parent = {start: None}
    expanded = 0
    visited = set()
    depth = {start: 0}
    while not openq.empty():
        n, _ = openq.pop_min()
        if n in visited:
            continue
        visited.add(n)
        expanded += 1
        if n == goal:
            t1 = time.perf_counter()
            path = reconstruct(parent, goal)
            return SearchResult(path, g[goal], expanded, depth[goal], (t1 - t0)*1000)
        for nbr in G.neighbors(n):
            w = G[n][nbr]['weight']
            tentative = g[n] + w
            if nbr not in g or tentative < g[nbr]:
                g[nbr] = tentative
                parent[nbr] = n
                depth[nbr] = depth[n] + 1
                openq.push_or_decrease(nbr, g[nbr])
    return SearchResult([], float('inf'), expanded, -1, (time.perf_counter() - t0)*1000)

# -----------------
# A*
# -----------------

def astar(G: nx.Graph, start: Any, goal: Any, h: Heuristic) -> SearchResult:
    t0 = time.perf_counter()
    openq = PriorityQueue()
    openq.push_or_decrease(start, h(start))
    g = {start: 0.0}
    parent = {start: None}
    depth = {start: 0}
    closed = set()
    expanded = 0
    while not openq.empty():
        n, _ = openq.pop_min()
        if n in closed:
            continue
        closed.add(n)
        expanded += 1
        if n == goal:
            t1 = time.perf_counter()
            path = reconstruct(parent, goal)
            return SearchResult(path, g[goal], expanded, depth[goal], (t1 - t0)*1000)
        for nbr in G.neighbors(n):
            w = G[n][nbr]['weight']
            tentative = g[n] + w
            if nbr not in g or tentative < g[nbr]:
                g[nbr] = tentative
                parent[nbr] = n
                depth[nbr] = depth[n] + 1
                f = g[nbr] + h(nbr)
                openq.push_or_decrease(nbr, f)
    return SearchResult([], float('inf'), expanded, -1, (time.perf_counter() - t0)*1000)

# -----------------
# Construcción de grafo de ejemplo
# -----------------

def build_sample_graph() -> Tuple[nx.Graph, Any, Any]:
    G = nx.Graph()
    # nodos con posiciones (x,y)
    coords: Dict[str, Position] = {
        'A': (0, 0), 'B': (2, 1), 'C': (4, 0),
        'D': (1, 3), 'E': (3, 3), 'F': (5, 2)
    }
    for n, pos in coords.items():
        G.add_node(n, pos=pos)
    def euclid(u,v):
        x1,y1 = G.nodes[u]['pos']
        x2,y2 = G.nodes[v]['pos']
        return math.hypot(x1-x2, y1-y2)
    # aristas (puedes ajustar para simular cierres/restricciones)
    edges = [
        ('A','B'), ('B','C'), ('A','D'), ('B','D'),
        ('B','E'), ('C','F'), ('D','E'), ('E','F')
    ]
    for u,v in edges:
        G.add_edge(u, v, weight=euclid(u,v))
    return G, 'A', 'F'

# -----------------
# Runner de experimentos (automatización comparativa)
# -----------------

def run_compare(G: nx.Graph, start: Any, goal: Any, heuristic_name: str = 'euclid', alpha: float = 1.2):
    results = []
    # UCS (baseline)
    r_ucs = ucs(G, start, goal)
    results.append(('UCS', 'h=0', r_ucs))
    # A* con h=0 (equivalente a UCS)
    r_h0 = astar(G, start, goal, h_zero(goal))
    results.append(('A*', 'h=0', r_h0))
    # A* euclidiana
    if heuristic_name == 'euclid':
        r_astar = astar(G, start, goal, h_euclid(G, goal))
        results.append(('A*', 'h=euclid', r_astar))
    # A* con sobreestimación controlada (para análisis)
    r_scaled = astar(G, start, goal, h_scaled(G, goal, alpha=alpha))
    results.append(('A*', f'h={alpha}*euclid', r_scaled))
    # Formato salida
    table = []
    for algo, hname, r in results:
        table.append({
            'algo': algo,
            'heuristic': hname,
            **r.as_dict()
        })
    return table


In [4]:
if __name__ == '__main__':
    G, s, t = build_sample_graph()
    table = run_compare(G, s, t, heuristic_name='euclid', alpha=1.3)
    for row in table:
        print(row)

{'algo': 'UCS', 'heuristic': 'h=0', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 6, 'depth': 3, 'runtime_ms': 0.15}
{'algo': 'A*', 'heuristic': 'h=0', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 6, 'depth': 3, 'runtime_ms': 0.022}
{'algo': 'A*', 'heuristic': 'h=euclid', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 5, 'depth': 3, 'runtime_ms': 0.019}
{'algo': 'A*', 'heuristic': 'h=1.3*euclid', 'path': ['A', 'B', 'C', 'F'], 'cost': 6.708203932499369, 'expanded': 4, 'depth': 3, 'runtime_ms': 0.015}


## Interpretación de resultados y comparación de algoritmos
A continuación se comentan las métricas obtenidas en el ejemplo para cada algoritmo:

## Resultados obtenidos
| Algoritmo | Heurística       | Camino encontrado       | Costo   | Nodos expandidos | Profundidad | Tiempo (ms) |
|-----------|------------------|------------------------|---------|------------------|-------------|-------------|
| UCS       | h=0              | A → B → C → F           | 6.7082  | 6                | 3           | 0.150       |
| A*        | h=0              | A → B → C → F           | 6.7082  | 6                | 3           | 0.022       |
| A*        | h=euclid         | A → B → C → F           | 6.7082  | 5                | 3           | 0.019       |
| A*        | h=1.3×euclid     | A → B → C → F           | 6.7082  | 4                | 3           | 0.015       |

---

**Interpretación:**
- Todas las configuraciones encontraron el mismo camino óptimo con costo ≈ 6.71.
- Usar una heurística informativa (euclidiana) redujo la cantidad de nodos expandidos y el tiempo de ejecución frente a UCS.
- La heurística escalada (1.3×euclid) aceleró aún más la búsqueda, aunque en grafos más complejos podría perder optimalidad.



## Respuestas a Preguntas Orientadoras

**1) ¿Cómo afecta la calidad de la heurística?**
> Una heurística más informativa (euclidiana) reduce el número de nodos expandidos y el tiempo de ejecución sin perder optimalidad. La heurística escalada (1.3×euclid) reduce aún más las expansiones, pero podría sacrificar optimalidad en casos más complejos.

**2) ¿Es pertinente la euclidiana?**
> Sí, porque el peso de las aristas se calculó como distancia euclidiana. Esto garantiza que la heurística sea admisible y consistente.

**3) ¿Flexibilidad ante nuevas rutas o restricciones?**
> El modelo permite añadir o quitar aristas y ajustar pesos fácilmente. Los algoritmos pueden recalcular rutas óptimas sin modificar la lógica interna.

**4) ¿Escalabilidad si el grafo crece 10× en tamaño?**
> UCS y A* seguirán funcionando, pero el tiempo y memoria crecerán. Una heurística de calidad (como la euclidiana) será clave para reducir expansiones. En grafos muy grandes se recomienda optimizar con técnicas como búsqueda bidireccional o preprocesamiento de rutas.

---

# Punto 2

## Pseudocodigo 

ALGORITMO Anchura(G, inicio, meta)
    frontera ← Cola()                    // FIFO
    frontera.encolar(inicio)
    explorados ← {inicio}
    padre ← diccionario vacío

    MIENTRAS frontera NO esté vacía HACER
        n ← frontera.desencolar()
        SI n = meta ENTONCES
            DEVOLVER reconstruir_camino(padre, n)
        FIN SI

        PARA CADA vecino EN G.adyacentes(n) HACER
            SI vecino NO está EN explorados ENTONCES
                explorados.agregar(vecino)
                frontera.encolar(vecino)
                padre[vecino] ← n
            FIN SI
        FIN PARA
    FIN MIENTRAS

    DEVOLVER fallo
FIN ALGORITMO

In [ ]:
## Codigo 

In [ ]:
# Punto 3

## Pseudocodigo

Entrada: G=(V,E), inicio s, destino t, weight(u,v) en minutos, h(n)
Salida: ruta óptima y tiempo total

1  open ← PriorityQueue()
2  open.push(s, key=h(s))
3  g[s] ← 0
4  parent[s] ← None
5  closed ← ∅
6  expanded ← 0
7  while open ≠ ∅:
8      (n, _) ← open.pop_min()
9      if n ∈ closed: continue
10     closed ← closed ∪ {n}
11     expanded ← expanded + 1
12     if n == t:
13         return Reconstruir(parent, t), g[t], expanded
14     for cada m ∈ vecinos(n):
15         costo_tent ← g[n] + weight(n,m)
16         if m ∉ g or costo_tent < g[m]:
17             g[m] ← costo_tent
18             parent[m] ← n
19             f_m ← g[m] + h(m)
20             open.push_or_decrease(m, f_m)
21 return ∅, ∞, expanded




In [ ]:
## Codigo

In [7]:
import networkx as nx
import math

# Construcción del grafo de la red de metro
def build_metro_graph():
    G = nx.Graph()
    coords = {
        'A': (0, 0),
        'B': (2, 0),
        'C': (4, 0),
        'D': (4, 2)
    }
    for station, pos in coords.items():
        G.add_node(station, pos=pos)
    edges = [
        ('A', 'B', 3),
        ('B', 'C', 4),
        ('C', 'D', 5)
    ]
    for u, v, time in edges:
        G.add_edge(u, v, weight=time)
    return G, 'A', 'D'

# Heurística basada en línea recta y velocidad promedio (40 km/h ≈ 0.667 km/min)
def h_metro(G, goal):
    vel = 0.667
    xg, yg = G.nodes[goal]['pos']
    def h(n):
        x, y = G.nodes[n]['pos']
        dist = math.hypot(x - xg, y - yg)
        return dist / vel
    return h

# Ejecución de A*
from heapq import heappush, heappop

def astar_metro(G, start, goal, h):
    openq = []
    heappush(openq, (h(start), start))
    g = {start: 0}
    parent = {start: None}
    expanded = 0
    closed = set()
    while openq:
        _, n = heappop(openq)
        if n in closed:
            continue
        closed.add(n)
        expanded += 1
        if n == goal:
            return reconstruct(parent, goal), g[goal], expanded
        for nbr in G.neighbors(n):
            time_cost = G[n][nbr]['weight']
            tentative = g[n] + time_cost
            if nbr not in g or tentative < g[nbr]:
                g[nbr] = tentative
                parent[nbr] = n
                heappush(openq, (g[nbr] + h(nbr), nbr))
    return [], float('inf'), expanded

def reconstruct(parent, goal):
    path = []
    n = goal
    while n is not None:
        path.append(n)
        n = parent.get(n)
    return path[::-1]

# Ejemplo de ejecución
G, s, t = build_metro_graph()
path, cost, expanded = astar_metro(G, s, t, h_metro(G, t))
print("Ruta:", path)
print("Tiempo total (min):", round(cost, 2))
print("Nodos expandidos:", expanded)

Ruta: ['A', 'B', 'C', 'D']
Tiempo total (min): 12
Nodos expandidos: 4


## Respuestas a Preguntas Orientadoras

# Punto 3

## Pseudocodigo

SUBALGORITMO Profundidad_Limitada(G, n, meta, límite)
    SI n = meta ENTONCES
        DEVOLVER reconstruir_camino(global_padre, n)
    FIN SI
    SI límite = 0 ENTONCES
        DEVOLVER "corte"                           // profundidad agotada
    FIN SI
    corte_ocurrió ← FALSO
    PARA CADA vecino EN G.adyacentes(n) HACER
        SI vecino NO visitado ENTONCES
            global_padre[vecino] ← n
            resultado ← Profundidad_Limitada(G, vecino, meta, límite-1)
            SI resultado = "corte" ENTONCES
                corte_ocurrió ← VERDADERO
            SINO SI resultado ≠ fallo ENTONCES
                DEVOLVER resultado                // se halló solución
            FIN SI
        FIN SI
    FIN PARA
    SI corte_ocurrió ENTONCES DEVOLVER "corte" SINO DEVOLVER fallo
FIN SUBALGORITMO


ALGORITMO ACMR_Profundidad_Limitada(Especie a, Especie b, límite L, DAG filogenia)
    ancestros_a ← Ancestros_Limitados(a, L, filogenia)
    ancestros_b ← Ancestros_Limitados(b, L, filogenia)
    comunes ← intersección(ancestros_a, ancestros_b)
    SI comunes vacía ENTONCES DEVOLVER "sin ancestro común dentro de L"
    DEVOLVER más_profundo(comunes)                 // profundidad mínima desde a y b
FIN ALGORITMO


## Codigo

## Respuestas a Preguntas Orientadoras

# Punto 4

## Pseudocodigo

ALGORITMO ACMR_Prof_Lim(a, b, L, grafo)
    // Obtener ancestros hasta L niveles
    A ← Ancestros_Limitados(a, L, grafo)
    B ← Ancestros_Limitados(b, L, grafo)

    comunes ← A ∩ B
    SI comunes vacío ENTONCES
        DEVOLVER "sin ancestro común hasta L"
    FIN SI

    DEVOLVER ancestro_más_profundo(comunes)
FIN ALGORITMO

SUBALGORITMO Ancestros_Limitados(nodo, L, grafo)
    conjunto ← {nodo}
    SI L = 0 ENTONCES
        DEVOLVER conjunto
    FIN SI

    PARA CADA padre EN grafo.predecesores(nodo) HACER
        conjunto ← conjunto ∪ Ancestros_Limitados(padre, L-1, grafo)
    FIN PARA

    DEVOLVER conjunto
FIN SUBALGORITMO


## Codigo

## Respuestas a Preguntas Orientadoras